# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902059


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:42,  1.45s/it]

Rendering models:  13%|█▎        | 4/31 [00:03<00:28,  1.04s/it]

Rendering models:  16%|█▌        | 5/31 [00:03<00:20,  1.25it/s]

Rendering models:  19%|█▉        | 6/31 [00:07<00:47,  1.91s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:14<00:00,  3.44it/s]

Rendering models:  94%|█████████▎| 29/31 [00:14<00:00,  3.26it/s]

Rendering models:  97%|█████████▋| 30/31 [00:15<00:00,  2.71it/s]

Rendering models: 100%|██████████| 31/31 [00:15<00:00,  2.99it/s]

equidad1                               0.000476
kayleebug2017                          0.002135
JessieDawn                             0.000545
ElisabethB                             0.006940
not-logged-in-8c0c44faf754d344b18a     0.008470
Thedunkmasta                           0.003275
not-logged-in-84be6415a703f6e6a7e1     0.000873
insert_name                            0.000650
Jboy2000                               0.031060
not-logged-in-f09819078c043556a4da     0.023178
Jessica_Korkmaz                        0.002382
not-logged-in-314d48142a3a122ba666     0.000754
xsara                                  0.002973
not-logged-in-f8c2f88355fba4282271     0.002237
alphamoonbase                          0.000572
pangeli5                               0.000482
aidenr                                 0.000390
not-logged-in-04ed9ae880f18762cd9e     0.015363
not-logged-in-687931fdaee12bdb17bd     0.103966
not-logged-in-90294de164951e098d0b     0.001463
not-logged-in-6ea7dd94a335e5238780     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())